In [1]:
import numpy as np
import sympy as sp
import math

In [2]:
set_of_nodes = []
set_of_elements = []
set_of_equations = []
set_of_variables = []

In [3]:
class node:
    def __init__(self, voltage):
        global set_of_nodes, set_of_variables
        if type(voltage) == int or type(voltage) == sp.core.symbol.Symbol:
            self.voltage = voltage
            self.str_input_flag = False
        elif type(voltage) == str:
            self.voltage = sp.symbols(voltage)
            self.str_input_flag = True
        self.neighbours_vsource = {}
        self.neighbours_isource = {}
        self.neighbours_impedance = {}
        set_of_nodes.append(self)
    
    
    def net_current(self):
        global set_of_equations
        balance = 0
        for i in self.neighbours_impedance:
            balance -= (self.voltage - self.neighbours_impedance[i].voltage)/i.value

        for i in self.neighbours_isource:
            if i.node_minus == self:
                balance -= i.value
            if i.node_plus == self:
                balance += i.value

        for i in self.neighbours_vsource:
            if i.node_minus == self:
                balance -= i.current
            if i.node_plus == self:
                balance += i.current
        self.balance = balance
        
class impedance:
    def __init__(self, n1, n2, value):
        global set_of_elements
        t = type(value)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.value = value
        elif t == str:
            self.value = sp.symbols(value)
        self.type = "impedance"
        self.node1 = n1
        self.node2 = n2
        n1.neighbours_impedance[self] = n2
        n2.neighbours_impedance[self] = n1
        set_of_elements.append(self)
        
class vsource:
    def __init__(self, n_minus, n_plus, value, current):
        global set_of_elements
        set_of_elements.append(self)
        self.type = "vsource"
        
        t = type(value)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.value = value
        elif t == str:
            self.value = sp.symbols(value)
        
        self.node_minus = n_minus
        self.node_plus = n_plus
        self.expr = self.node_plus.voltage - self.node_minus.voltage - self.value
        
        t = type(current)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.current = current
            self.str_input_flag = False
        elif t == str:
            self.current = sp.symbols(current)
            self.str_input_flag = True
            
        n_minus.neighbours_vsource[self] = n_plus
        n_plus.neighbours_vsource[self] = n_minus
        
class vcvs:
    def __init__(self, n_in_minus, n_in_plus, n_out_minus, n_out_plus, gain, current):
        global set_of_elements
        set_of_elements.append(self)
        self.type = "vsource"
        
        t = type(gain)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.gain = gain
        elif t == str:
            self.gain = sp.symbols(gain)
        
        self.node_minus = n_out_minus
        self.node_plus = n_out_plus
        self.expr = self.node_plus.voltage - self.node_minus.voltage - self.gain*(n_in_plus.voltage - n_in_minus.voltage)
        
        t = type(current)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.current = current
            self.str_input_flag = False
        elif t == str:
            self.current = sp.symbols(current)
            self.str_input_flag = True
            
        n_out_minus.neighbours_vsource[self] = n_out_plus
        n_out_plus.neighbours_vsource[self] = n_out_minus
        
class ccvs:
    def __init__(self, n_in_minus, n_in_plus, element, n_out_minus, n_out_plus, tximp, current):
#         remember to raise error if n_in_plus and n_in_minus aren't connected by element
        global set_of_elements
        set_of_elements.append(self)
        self.type = "vsource"
        
        t = type(tximp)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.tximp = tximp
        elif t == str:
            self.tximp = sp.symbols(tximp)
        
        if element.type == "impedance":
            if element not in n_in_minus.neighbours_impedance or n_in_minus != n_in_plus.neighbours_impedance[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            self.value = self.tximp*(n_in_plus.voltage - n_in_minus.voltage)/(element.value)
        if element.type == "vsource":
            if element not in n_in_minus.neighbours_vsource or n_in_minus != n_in_plus.neighbours_vsource[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            if element.node_plus == n_in_minus:
                self.value = self.tximp*element.current
            if element.node_plus == n_in_plus:
                self.value = -self.tximp*element.current
        if element.type == "isource":
            if element not in n_in_minus.neighbours_isource or n_in_minus != n_in_plus.neighbours_isource[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            if element.node_plus == n_in_minus:
                self.value = self.tximp*element.value
            if element.node_plus == n_in_plus:
                self.value = -self.tximp*element.value
               
        self.node_minus = n_out_minus
        self.node_plus = n_out_plus
        self.expr = self.node_plus.voltage - self.node_minus.voltage - self.value
        
        t = type(current)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.current = current
            self.str_input_flag = False
        elif t == str:
            self.current = sp.symbols(current)
            self.str_input_flag = True
            
        n_out_minus.neighbours_vsource[self] = n_out_plus
        n_out_plus.neighbours_vsource[self] = n_out_minus
    
class isource:
    def __init__(self, n_minus, n_plus, value):
        global set_of_elements
        set_of_elements.append(self)
        self.type = "isource"
        
        t = type(value)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.value = value
        elif t == str:
            self.value = sp.symbols(value)
        
        self.node_minus = n_minus
        self.node_plus = n_plus
        n_minus.neighbours_isource[self] = n_plus
        n_plus.neighbours_isource[self] = n_minus
        
class vccs:
    def __init__(self, n_in_minus, n_in_plus, n_out_minus, n_out_plus, Gm):
        global set_of_elements
        set_of_elements.append(self)
        t = type(Gm)
        self.type = "isource"
        
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.Gm = Gm
        elif t == str:
            self.Gm = sp.symbols(Gm)
        
        self.node_minus = n_out_minus
        self.node_plus = n_out_plus
        self.value = self.Gm * (n_in_plus.voltage - n_in_minus.voltage)
        n_out_minus.neighbours_isource[self] = n_out_plus
        n_out_plus.neighbours_isource[self] = n_out_minus

class cccs:
    def __init__(self, n_in_minus, n_in_plus, element, n_out_minus, n_out_plus, gain):
#         remember to raise error if n_in_plus and n_in_minus aren't connected by element
        global set_of_elements
        set_of_elements.append(self)
        self.type = "isource"
        
        t = type(gain)
        if t == int or t == sp.core.symbol.Symbol or t == sp.core.add.Add or t == sp.core.mul.Mul:
            self.gain = gain
        elif t == str:
            self.gain = sp.symbols(gain)
        
        if element.type == "impedance":
            if element not in n_in_minus.neighbours_impedance or n_in_minus != n_in_plus.neighbours_impedance[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            self.value = self.gain*(n_in_plus.voltage - n_in_minus.voltage)/(element.value)
        if element.type == "vsource":
            if element not in n_in_minus.neighbours_vsource or n_in_minus != n_in_plus.neighbours_vsource[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            if element.node_plus == n_in_minus:
                self.value = self.gain*element.current
            if element.node_plus == n_in_plus:
                self.value = -self.gain*element.current
        if element.type == "isource":
            if element not in n_in_minus.neighbours_isource or n_in_minus != n_in_plus.neighbours_isource[element]:
                raise Exception("The two nodes aren't connected by the element specified.")
            if element.node_plus == n_in_minus:
                self.value = self.gain*element.value
            if element.node_plus == n_in_plus:
                self.value = -self.gain*element.value
        
        self.node_minus = n_out_minus
        self.node_plus = n_out_plus
        n_out_minus.neighbours_isource[self] = n_out_plus
        n_out_plus.neighbours_isource[self] = n_out_minus
        
class MOSFET_small_signal:
    def __init__(self, gate, source, drain, name, ideal):
        name = str(name)
        globals()["gm_"+name] = vccs(n_in_minus=source,
                                     n_in_plus=gate,
                                     n_out_minus=drain,
                                     n_out_plus=source,
                                     Gm = "gm_"+name)
        if not ideal:
            globals()["ro_"+name] = impedance(n1=source, n2=drain, value="ro_"+name)
        
def disconnect(element):
    global set_of_variables
    set_of_elements.remove(element)
    if element.type == "impedance":
        element.node1.neighbours_impedance.pop(element)
        element.node2.neighbours_impedance.pop(element)
    if element.type == "vsource":
        element.node_minus.neighbours_vsource.pop(element)
        element.node_plus.neighbours_vsource.pop(element)
    if element.type == "isource":
        element.node_minus.neighbours_isource.pop(element)
        element.node_plus.neighbours_isource.pop(element) 
    
def solve_ckt():
    global set_of_equations, set_of_nodes, set_of_variables, set_of_elements
    set_of_equations = []
    set_of_variables = []
    for i in set_of_nodes:
        i.net_current()
        set_of_equations.append(i.balance)
        if i.str_input_flag:
            set_of_variables.append(i.voltage)
    for i in set_of_elements:
        if i.type == "vsource":
            set_of_equations.append(i.expr)
            if i.str_input_flag:
                set_of_variables.append(i.current)
    sol_set = sp.linsolve(set_of_equations, set_of_variables)
#     print(set_of_variables)
    return sol_set

def display_solns():
    a = solve_ckt()
    print("--------------------")
    print("---SOLVED CIRCUIT---")
    print("--------------------")
    print("\n")
    
    for i in range(len(set_of_variables)):
#         print(sp.latex(sp.Eq(set_of_variables[i], a.args[0][i])))
        sp.pprint(sp.Eq(set_of_variables[i], a.args[0][i]))
        print("\n")

def clear_ckt():
    global set_of_equations, set_of_nodes, set_of_variables, set_of_elements
    for i in set_of_nodes:
        del i
    for i in set_of_elements:
        del i
    set_of_equations = []
    set_of_nodes = []
    set_of_variables = []
    set_of_elements = []
    

In [24]:
# if "value" is a string, convert it into sympy symbols
# else, keep as integer
# make list of all equations that have to be satisfied.
# ^^ this includes KCL at all nodes and KVL at all voltage sources
# everytime a new node is created, add it to a list of nodes in the ckt.
# KCL has to be satisfied for each element in this list.
# everytime a new ckt element is created, add it to a list of elements in the ckt.
# everytime a new variable is added, it has to be also be added to the list of variables to be solved for
# include support for controlled sources
# throw error if n_in_plus and n_in_minus for ccxs are not connected by element
# include support for xfmr
# include support for transistors (in saturation?)

# organize code!! Make classes neat and tidy!!

# for vsource, V(plus node) - V(minus node) = value
# for isource, current flows from minus node to plus node

In [4]:
clear_ckt()

n1 = node(0)
n2 = node("V123")
Vin = vsource(n1, n2, "V_{in}", "I_{in}")
R1 = impedance(n1, n2, "R")
n3 = node(0)
n4 = node("Vx")
R2 = impedance(n3, n4, "Rx")
V2 = ccvs(n2, n1, R1, n3, n4, 10, "I_{out}")
display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


V₁₂₃ = V_{in}


     -10⋅V_{in} 
Vx = ───────────
          R     


         V_{in}
I_{in} = ──────
           R   


          -10⋅V_{in} 
I_{out} = ───────────
              R⋅Rx   




In [11]:
clear_ckt()
g = node("V_in")
s = node(0)
d = node("V_out")
Vin = vsource(s, g, "V_{in}", "I_{in}")
R = impedance(d, s, "R")
M1 = MOSFET_small_signal(g, s, d, "1", True)
Rb = impedance(g, d, "Rb")
display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


Vᵢₙ = V_{in}


       R⋅V_{in}⋅(-Rb⋅gm₁ + 1)
Vₒᵤₜ = ──────────────────────
               R + Rb        


         V_{in}⋅(R⋅gm₁ + 1)
I_{in} = ──────────────────
               R + Rb      




In [5]:
clear_ckt()
s, C = sp.symbols('s, C')
n1 = node(0)
n2 = node("Vx")
n3 = node("V_{in}")
C1 = impedance(n3, n2, 1/(s*C))
R = impedance(n2, n1, "R")
vin = vsource(n1, n3, "Vin", "Iin")
display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


     C⋅R⋅Vin⋅s
Vx = ─────────
     C⋅R⋅s + 1


V_{in} = Vin


       C⋅Vin⋅s 
Iin = ─────────
      C⋅R⋅s + 1




In [6]:
clear_ckt()
n0 = node(0)
n1 = node("V1")
Vin = vsource(n0, n1, "V_{in}", "I_{in}")
n_inv = node("V_{out}")
amp = vcvs(n_inv, n1, n0, n_inv, 1000, "I_{out}")
display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


V₁ = V_{in}


          1000⋅V_{in}
V_{out} = ───────────
              1001   


I_{in} = 0


I_{out} = 0




In [10]:
clear_ckt()

gnd = node(0)
n1 = node("Vx")
n2 = node("Vy")
n3 = node("Vz")
R1 = impedance(n2, n1, "R1")
R2 = impedance(n2, n3, "R2")
R3 = impedance(n3, gnd, "R3")
I1 = isource(gnd, n1, "I_{1}")
V1 = vsource(n2, gnd, "V_{in}", "I?")

display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


Vx = I_{1}⋅R₁ - V_{in}


Vy = -V_{in}


     -R₃⋅V_{in} 
Vz = ───────────
       R₂ + R₃  


     I_{1}⋅R₂ + I_{1}⋅R₃ + V_{in}
I? = ────────────────────────────
               R₂ + R₃           




In [8]:
clear_ckt()
s, C1, C2, C3 = sp.symbols('s, C1, C2, C3')
gnd = node(0)
n1 = node("V1")
n2 = node("V2")
n3 = node("V3")
n_in = node("V_{in}")
vin = vsource(gnd, n_in, "Vin", "Iin")
M1 = MOSFET_small_signal(gate=n_in, source=gnd, drain=n1, name=1, ideal=True)
M2 = MOSFET_small_signal(gate=n1, source=gnd, drain=n2, name=2, ideal=True)
M3 = MOSFET_small_signal(gate=n2, source=n1, drain=n3, name=3, ideal=True)
c1 = impedance(n1, gnd, 1/(s*C1))
c2 = impedance(n2, gnd, 1/(s*C2))
c3 = impedance(n3, gnd, 1/(s*C3))
display_solns()

--------------------
---SOLVED CIRCUIT---
--------------------


             -C₂⋅Vin⋅gm₁⋅s        
V₁ = ─────────────────────────────
            2                     
     C₁⋅C₂⋅s  + C₂⋅gm₃⋅s + gm₂⋅gm₃


              Vin⋅gm₁⋅gm₂         
V₂ = ─────────────────────────────
            2                     
     C₁⋅C₂⋅s  + C₂⋅gm₃⋅s + gm₂⋅gm₃


          -Vin⋅gm₁⋅gm₃⋅(C₂⋅s + gm₂)      
V₃ = ────────────────────────────────────
          ⎛       2                     ⎞
     C₃⋅s⋅⎝C₁⋅C₂⋅s  + C₂⋅gm₃⋅s + gm₂⋅gm₃⎠


V_{in} = Vin


Iin = 0




In [9]:
sp.Eq(set_of_variables[2], solve_ckt().args[0][2])

Eq(V3, -Vin*gm_1*gm_3*(C2*s + gm_2)/(C3*s*(C1*C2*s**2 + C2*gm_3*s + gm_2*gm_3)))